<a href="https://colab.research.google.com/github/meghanak-27/skinDisease/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
!git clone https://github.com/meghanak-27/skinDisease


fatal: destination path 'skinDisease' already exists and is not an empty directory.


In [92]:

import pandas as pd
# from PIL import Image
import cv2
import numpy as np
import os


class driveFile:

  def __init__(self, mountpath):
    import os
    self.path = mountpath
    from google.colab import drive
    drive.mount(self.path, force_remount=True)
  def dataload(self,folder_path):
    img=[]
    label=[]
    import os
    self.path1=os.path.join(self.path,folder_path)

    if os.path.exists(self.path1):
      for i in os.listdir(self.path1):
        self.dis_path=os.path.join(self.path1,i)
        # /content/drive/MyDrive/train_set/BA- cellulitis
        if os.path.exists(self.dis_path) and '.DS_Store' not in self.dis_path:
          for j in os.listdir(self.dis_path):
           if j.endswith(('.jpg', '.png', '.jpeg')):
              self.img_path = os.path.join(self.dis_path, j)
              loaded_img= self.imgload(self.img_path)
              if loaded_img is not None:
                img.append(loaded_img)
                label.append(i)
              else:
                 print("Skipping non-image file:", j)
          else:
              print("Skipping invalid path or .DS_Store:", self.dis_path)
              continue
      return np.array(img), np.array(label)
    else:
      return "patherror","patherror"



  def imgload(self,imgpath):
    img=cv2.imread(imgpath)
    if img is None:
      print(f"Failed to load image at {imgpath}")
      return None
    img=cv2.resize(img,(20,20))
    # v imp, usually 224*224 but here 20*20 "downsampling or using a smaller image resolution before flattening it."
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img.astype('float32')/255.0
    img_array=np.array(img)
    return img_array

In [93]:
drive=driveFile('/content/drive')
images,labels=drive.dataload('MyDrive/train_set')

# 1d


Mounted at /content/drive
Skipping invalid path or .DS_Store: /content/drive/MyDrive/train_set/BA- cellulitis
Skipping invalid path or .DS_Store: /content/drive/MyDrive/train_set/FU-athlete-foot


In [94]:
class train:
  def __init__(self,X,w,b,Y,learning_rate,batch_size,epochs):
    self.X=X
    self.w=w
    self.b=b
    self.y=Y
    self.learning_rate=learning_rate
    self.batch_size=batch_size
    self.epoch=epochs
    self.training()

  def perceptron(self,mini_x):
    y_linearop=np.dot(mini_x,self.w)+self.b
        #unbounded
    return y_linearop

  def sigmoid(self,mini_X):#activatn fn
    y_predprob=1/(1+np.exp(-self.perceptron(mini_X)))
    return y_predprob

  def binary_cross_entropy(self,mini_x,mini_y):
     self.y_pred=self.sigmoid(mini_x)
     self.y_pred = np.clip(self.y_pred, 1e-10, 1 - 1e-10)  # Prevent extreme values
     self.loss=-np.mean((mini_y*np.log(self.y_pred))+((1-mini_y)*np.log(1-self.y_pred)))
     return self.loss

  def backward_prop(self,mini_X,mini_y):
    dloss_dy_pred=-(mini_y/self.y_pred)+((1-mini_y)/(1-self.y_pred))
    dsigm_dlinearoutput=self.y_pred*(1-self.y_pred)
    #bcz dsigmoid/dz=sigmod-sigmoid^2
    dloss_dlinearoutput=dloss_dy_pred*dsigm_dlinearoutput
    # (f(g(x)))`=f`(g(x))g'(x)

    self.dw = np.dot(mini_X.T, dloss_dlinearoutput) / mini_X.shape[0]
    self.db = np.mean(dloss_dlinearoutput)
    self.update_para()


  def update_para(self):
    self.w -= self.learning_rate * self.dw
    self.b -= self.learning_rate * self.db

  def training(self):
    num_images=self.X.shape[0]
    for epoch in range(self.epoch):
      # shuffling
      index=np.arange(num_images)
      np.random.shuffle(index)
      self.X=self.X[index]
      self.y=self.y[index]
      for i in range(0,num_images,self.batch_size):
        if(i+self.batch_size>num_images):
          X_batch=self.X[i:i+num_images]
          y_batch=self.y[i:i+num_images]
        else:
          X_batch=self.X[i:i+self.batch_size]
          y_batch=self.y[i:i+self.batch_size]

        loss=self.binary_cross_entropy(X_batch,y_batch)
        print(f"loss {loss} for mini batch {i // self.batch_size+1} in epoch: {epoch}")
        self.backward_prop(X_batch,y_batch)



In [95]:
print(images.shape)
print(labels.shape)
images=np.array([img.flatten() for img in images])
# images=np.array([img.reshape(-1,1) for img in images])
print(images.shape)
labels=np.array([1 if label=='BA- cellulitis' else 0 for label in labels])
# ba-cellulitis 1, foot 0


# # imgs=[img1,
# # img2,
# # img3,
# #
# #
# # img200]



(197, 20, 20)
(197,)
(197, 400)


1D Array (Shape (400,)):

In a 1D array, NumPy doesn't explicitly distinguish between rows and columns. It's just a sequence of 400 elements along a single axis (the x-axis).
When you perform element-wise operations, such as scalar multiplication, it operates directly on each element because NumPy sees it as a single dimension (or vector).
2D Array (Shape (400, 1)):

Here, NumPy treats it as a column vector. The shape (400, 1) has two axes: the first axis (rows) has 400 elements, and the second axis (columns) has 1 element.
Even though there is only one column, it's a 2D structure. When performing element-wise operations, NumPy knows it needs to perform operations on a 2D grid (or matrix).

In [96]:
class model(train):
  def __init__(self,X,w,b,Y,learning_rate,batch_size,epochs):
    super().__init__(X,w,b,Y,learning_rate,batch_size,epochs)
  def evaluate(self,X,y):
    print(f"loss={self.binary_cross_entropy(X,y)}")
  def prediction(self,X,y):
    pred=self.sigmoid(X)
    pred_op=(pred>=0.5).astype(int)
    return pred_op
  def accuracy(self,X,y):
    pred_op=self.prediction(X,y)
    predictions=np.mean(pred_op==y)
    acc=np.mean(predictions)
    print(acc)


In [ ]:
w=np.random.randn(images.shape[1],)*0.01
b=0
learning_rate=0.5
epochs=10
bs=8
mymodel=model(images,w,b,labels,learning_rate,bs,epochs)

In [98]:
testimages,testlabels=drive.dataload('MyDrive/test_set')

Skipping invalid path or .DS_Store: /content/drive/MyDrive/test_set/BA- cellulitis
Skipping invalid path or .DS_Store: /content/drive/MyDrive/test_set/FU-athlete-foot


In [99]:
testimages=np.array([img.flatten() for img in testimages])
testlabels=np.array([1 if label=='BA- cellulitis' else 0 for label in testlabels])
mymodel.accuracy(testimages,testlabels)


0.5076923076923077
